# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [8]:
import numpy as np
import os
from cv2 import imread,resize,split
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [9]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [10]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        
        shear_range=0.5,
        zoom_range=0.5,
        rotation_range=20,
        width_shift_range=0.4,
        height_shift_range=0.4
        )

In [12]:
def generator(source_path, folder_list, batch_size,percentage_data,train = False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    folder_list = folder_list[:int(percentage_data*len(folder_list))]
    img_idx = [i for i in range(30)if i%2 == 0]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)        
        num_batches = int(len(folder_list)/batch_size) 
        #print("Number batches {}" .format(num_batches))
        remaining_folders = len(folder_list) - num_batches*batch_size
        #print(len(folder_list))
        for batch in range(num_batches): # we iterate over the number of batches            
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size               
                if train and np.random.randn(1)[0] > 0:
                    augmentation = True
                    #print("Augmenting this video")
                    transform_parameters = train_datagen.get_random_transform((100,100), seed=None)
                    
                else:
                    #print("Not augmenting")
                    augmentation = False
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
    
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    w,h,c = image.shape 
                    image = image[int((w/2-0.4*w)):int((w/2+0.4*w)) ,int((h/2-0.4*h)):int((h/2+0.4*h))]
                    image = resize(image,(100,100))
                    if augmentation:
                        #print(transform_parameters)
                        image = train_datagen.apply_transform(image, transform_parameters)
                        
#                     if train and np.random.randn(1)[0] > 0:
#                         image = train_datagen.random_transform(image)
                    b, g, r = split(image)
                    batch_data[folder,idx,:,:,0] =   (b - np.percentile(b,5)) / (np.percentile(b,95) - np.percentile(b,5))
                    batch_data[folder,idx,:,:,1] =   (g - np.percentile(g,5)) / (np.percentile(g,95) - np.percentile(g,5))
                    batch_data[folder,idx,:,:,2] =   (r - np.percentile(r,5)) / (np.percentile(r,95) - np.percentile(r,5))

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        if remaining_folders!=0:
            print("Remaining folders are {}" .format(remaining_folders))
            for folder in range(remaining_folders):
                imgs = os.listdir(source_path + '/' + t[folder + (batch_size*num_batches)].split(';')[0])
                remaining_data = np.zeros((remaining_folders,len(img_idx),100,100,3))
                remaining_labels = np.zeros((remaining_folders,5))
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    w,h,c = image.shape
                    image = image[int((w/2-0.4*w)):int((w/2+0.4*w)) ,int((h/2-0.4*h)):int((h/2+0.4*h))]
                    image = resize(image,(100,100))
#                     if augmentation:
#                         #print(transform_parameters)
#                         image = train_datagen.apply_transform(image, transform_parameters)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    b, g, r = split(image)

                    remaining_data[folder,idx,:,:,0] = (b - np.percentile(b,5)) / (np.percentile(b,95) - np.percentile(b,5))
                    remaining_data[folder,idx,:,:,1] = (g - np.percentile(g,5)) / (np.percentile(g,95) - np.percentile(g,5))
                    remaining_data[folder,idx,:,:,2] = (r - np.percentile(r,5)) / (np.percentile(r,95) - np.percentile(r,5))


                remaining_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield remaining_data, remaining_labels
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [13]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = int(len(train_doc))
print('# training sequences =', num_train_sequences)
num_val_sequences = int(len(val_doc))
print('# validation sequences =', num_val_sequences)
num_epochs = 70 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 70


In [14]:
train_generator = generator(train_path, train_doc, batch_size,1,True)
val_generator = generator(val_path, val_doc, batch_size,1)

In [15]:
array,labels = train_generator.__next__()
print(array.shape ,labels.shape)

Source path =  Project_data/train ; batch size = 32
(32, 15, 100, 100, 3) (32, 5)


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [16]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import regularizers

model = Sequential()
model.add(Conv3D(8, (3,3,3),  input_shape=(15,100,100,3) ,padding='same' ,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 2, 2)))
model.add(Dropout(0.2))

model.add(Conv3D(16, (3,3,3), padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 2, 2)))
model.add(Dropout(0.2))

model.add(Conv3D(32, (3,3,3), padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv3D(64, (3,3,3), padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2)))
model.add(Dropout(0.2))


model.add(Conv3D(128, (3,3,3), padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(3,3,3), strides=(1, 2, 2)))
model.add(Dropout(0.5))

# model.add(Conv3D(256, (3,3,3),padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 2, 2)))

# model.add(Conv3D(1024, (1,1,1), activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv3D(128, (1,1,1), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# model.add(BatchNormalization())
# model.add(Conv3D(128, (2,2,2), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 1, 1)))
# model.add(BatchNormalization())
# model.add(Conv3D(256, (2,2,2), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 1, 1)))

model.add(Flatten())
model.add(Dense(128,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))

model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax' ,kernel_regularizer=regularizers.l2(0.01)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 100, 100, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 14, 50, 50, 8)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 50, 50, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 14, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 50, 50, 16)    64        
__________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [17]:
#optimiser = optimizers.SGD(lr=0.001,momentum=0.9)#write your optimizer
optimiser = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [18]:
train_generator = generator(train_path, train_doc, batch_size,1,True)
val_generator = generator(val_path, val_doc, batch_size,1)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001) # write the REducelronplateau code here
callbacks_list = [checkpoint,LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
#model.load_weights("model_init_2020-02-0904_12_53.123999/model-00060-1.32184-0.73906-1.20795-0.78000.h5", by_name=False)

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                     validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0 ,callbacks = callbacks_list)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/70
Remaining folders are 4
Remaining folders are 4
21/21 [==============================] - 143s 7s/step - loss: 16.1657 - categorical_accuracy: 0.2291 - val_loss: 16.1210 - val_categorical_accuracy: 0.1700

Epoch 00001: val_loss improved from inf to 16.12101, saving model to model_init_2020-02-1001_12_36.037942/model-00001-16.20597-0.22926-16.12101-0.17000.h5
Epoch 2/70
21/21 [==============================] - 49s 2s/step - loss: 15.4986 - categorical_accuracy: 0.2788 - val_loss: 15.1639 - val_categorical_accuracy: 0.1600

Epoch 00002: val_loss improved from 16.12101 to 15.16387, saving model to model_init_2020-02-1001_12_36.037942/model-00002-15.53037-0.27903-15.16387-0.16000.h5
Epoch 3/70
Remaining folders are 4
21/21 [==============================] - 51s 2s/step - loss: 14.4180 - categorical_accuracy: 0.2688 - val_loss: 13.3402 - val_categorical_accuracy: 0.2900

Epoch 000

#### Results:
Epochs : 60
Model size : 48 MB
Training accuracy - 81
Validation accuracy - 79